In [ ]:
running_as_notebook = False  # Manually set this to True if you are a human (see #13862).

# JointSliders

Running this cell should allow you to control the joints of the IIWA in drake visualizer.

In [ ]:
import numpy as np

from IPython.display import display
from ipywidgets import ToggleButton

from pydrake.common import FindResourceOrThrow
from pydrake.geometry import ConnectDrakeVisualizer, SceneGraph
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.systems.analysis import Simulator                                      
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.rendering import MultibodyPositionToGeometryPose

from pydrake.multibody.jupyter_widgets import JointSliders

builder = DiagramBuilder()
scene_graph = builder.AddSystem(SceneGraph())
plant = MultibodyPlant(time_step=0.0)
plant.RegisterAsSourceForSceneGraph(scene_graph)
Parser(plant, scene_graph).AddModelFromFile(FindResourceOrThrow(
    "drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf"))
plant.Finalize()

# Add sliders to set positions of the joints.
sliders = builder.AddSystem(JointSliders(robot=plant))
to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
builder.Connect(sliders.get_output_port(0), to_pose.get_input_port())
builder.Connect(
    to_pose.get_output_port(),
    scene_graph.get_source_pose_port(plant.get_source_id()))

ConnectDrakeVisualizer(builder, scene_graph)

# Make the diagram and run it.
diagram = builder.Build()
simulator = Simulator(diagram)

if running_as_notebook:  
    simulator.set_target_realtime_rate(1.0)
    stop_button = ToggleButton(value=False, description='Stop Simulation')
    display(stop_button)
    while not stop_button.value:
        simulator.AdvanceTo(simulator.get_context().get_time() + 5.0)
    stop_button.value = False
else:  # running as a test.
    simulator.AdvanceTo(0.1)
